In [2]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

In [3]:
# load gemma2 tokenizer from huggingface

from transformers import AutoTokenizer

gemma_tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")



/Users/alibayram/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
import pandas as pd
# get string, int pairs from gemma_tokenizer.vocab

gemma_vocab = gemma_tokenizer.get_vocab()

vocab_df = pd.DataFrame(gemma_vocab.items(), columns=['token', 'id'])
vocab_df = vocab_df.sort_values('id')
vocab_df[1000:1040]

,token,id
109689,ew,1000
136122,bu,1001
57330,ark,1002
157669,}),1003
15617,▁res,1004
185986,▁ag,1005
57367,▁tra,1006
113487,▁cont,1007
60968,▁et,1008
137825,▁some,1009


In [5]:
gemma_tokenizer.vocab_size

256000

In [6]:
vocab = {}

unk_token = '<unk>'
spl_tokens = ['<bos>', '<eos>', '<unk>', '<pad>', '<start_of_turn>', '<end_of_turn>']

vocab['<pad>'] = 0
vocab['<bos>'] = 1
vocab['<eos>'] = 2
vocab['<unk>'] = 3
vocab['<mask>'] = 4
vocab['<2mass>'] = 5
vocab['[@BOS@]'] = 6
for i in range(50):
    vocab[f'<unused{i}>'] = i + 7
vocab['<start_of_turn>'] = 57
vocab['<end_of_turn>'] = 58

vocab

#spl_tokens

{'<pad>': 0,
 '<bos>': 1,
 '<eos>': 2,
 '<unk>': 3,
 '<mask>': 4,
 '<2mass>': 5,
 '[@BOS@]': 6,
 '<unused0>': 7,
 '<unused1>': 8,
 '<unused2>': 9,
 '<unused3>': 10,
 '<unused4>': 11,
 '<unused5>': 12,
 '<unused6>': 13,
 '<unused7>': 14,
 '<unused8>': 15,
 '<unused9>': 16,
 '<unused10>': 17,
 '<unused11>': 18,
 '<unused12>': 19,
 '<unused13>': 20,
 '<unused14>': 21,
 '<unused15>': 22,
 '<unused16>': 23,
 '<unused17>': 24,
 '<unused18>': 25,
 '<unused19>': 26,
 '<unused20>': 27,
 '<unused21>': 28,
 '<unused22>': 29,
 '<unused23>': 30,
 '<unused24>': 31,
 '<unused25>': 32,
 '<unused26>': 33,
 '<unused27>': 34,
 '<unused28>': 35,
 '<unused29>': 36,
 '<unused30>': 37,
 '<unused31>': 38,
 '<unused32>': 39,
 '<unused33>': 40,
 '<unused34>': 41,
 '<unused35>': 42,
 '<unused36>': 43,
 '<unused37>': 44,
 '<unused38>': 45,
 '<unused39>': 46,
 '<unused40>': 47,
 '<unused41>': 48,
 '<unused42>': 49,
 '<unused43>': 50,
 '<unused44>': 51,
 '<unused45>': 52,
 '<unused46>': 53,
 '<unused47>': 54,
 '<un

In [1]:
predefined_words = []
with open("veri/first_12500_titled.txt") as f:
    for line in f:
        # remove newline character
        if line.strip() != "":
          predefined_words.append(line.strip())
predefined_words[:10]

['a', 'A', 'aba', 'Aba', 'abadi', 'Abadi', 'abaküs', 'Abaküs', 'aban', 'Aban']

In [48]:
ekler = [
    "lık", "lik", "luk", "lük",
    "lı", "li", "lu", "lü",
    "sız", "siz", "suz", "süz",
    "cı", "ci", "cu", "cü",
    "çı", "çi", "çu", "çü",
    "cık", "cik", "cuk", "cük",
    "çık", "çik", "çuk", "çük",
    "ca", "ce",
    "ça", "çe",
    "daş", "deş",
    "taş", "teş",
    "ncı", "nci", "ncu", "ncü",
    "ar", "er",
    "şar", "şer",
    "sal", "sel",
    "tı", "ti", "tu", "tü",
    "aç", "eç",
    "ak", "ek",
    "an", "en",
    "cıl", "cil", "cul", "cül",
    "çıl", "çil", "çul",
    "cileyin",
    "ç",
    "gil", "gül", "kıl", "kil",
    "ge", "ka",
    "kan", "ken",
    "kek",
    "man", "men",
    "la",
    "lak", "lek",
    "layın", "leyin",
    "msı", "msi", "msu",
    "mtırak",
    "rak", "rek",
    "sak", "sek",
    "sı", "si", "su", "sü",
    "şın", "şin",
    "t",
    "z",
    "la", "le",
    "al", "el",
    "l",
    "a", "e",
    "ar", "er",
    "da", "de",
    "k",
    "kır", "kir", "kur", "kür",
    "msa", "mse",
    "r",
    "sa", "se",
    "l",
    "ma", "me",
    "n",
    "r",
    "t",
    "ş",
    "dır", "dir", "dur", "dür",
    "tır", "tir", "tur", "tür",
    "a", "e",
    "ala", "ele",
    "ar", "er",
    "ı", "ü",
    "k",
    "k",
    "p",
    "a", "e",
    "ağan", "eğen",
    "ak", "ek",
    "alak", "elek",
    "amak", "emek",
    "anak", "enek",
    "ca", "ce",
    "ç",
    "aç", "eç",
    "dı", "di", "du", "dü",
    "tı", "ti", "tu", "tü",
    "ga", "ge",
    "gaç", "geç", "kaç",
    "gıç", "giç", "guç",
    "maca", "mece",
    "maç", "meç",
    "man", "men",
    "mık", "mik", "muk",
    "n",
    "t",
    "a", "e",
    "ga", "ge",
    "gıç", "giç", "guç",
    "n",
    "t",
    "lar", "ler", # Çokluk eki
    "mı", "mi", "mu", "mü", # Soru eki
    "m", "n", "ı", "i", "u", "ü", "sı", "si", "su", "sü", # İyelik ekleri
    "da", "de", "ta", "te", # Bulunma hali eki
    "dan", "den", "tan", "ten", # Ayrılma hali eki
    "ın", "in", "un", "ün", "nın", "nin", "nun", "nün", # İlgi hali eki
    "y", # Koruyucu ünsüz
    "n", # Koruyucu ünsüz
    "ki", # Aitlik eki
    "yor", "makta", "mede", # Şimdiki zaman ekleri
    "acak", "ecek", "acağ", "eceğ", # Gelecek zaman ekleri
    "r", "ar", "er", # Geniş zaman ekleri
    "malı", "meli", # Gereklilik kipi eki
    "sa", "se", # Dilek-şart kipi eki
    "a", "e", # İstek kipi eki
    "dı", "di", "du", "dü", "tı", "ti", "tu", "tü", # Görülen geçmiş zaman eki
    "mış", "miş", "muş", "müş", # Öğrenilen geçmiş zaman eki
    "im", "sin", "dir", "iz", "siniz", "dirler", # Ek-fiilin şimdiki/geniş zaman eki
    "dı", "di", "du", "dü", "tı", "ti", "tu", "tü", # Ek-fiilin görülen geçmiş zaman eki
    "mış", "miş", "muş", "müş", # Ek-fiilin öğrenilen geçmiş zaman eki
    "sa", "se", # Ek-fiilin şart eki
    "yacak", "yecek", # Katmerli birleşik çekim eki
    "miş", "miş", "muş", "müş", # Katmerli birleşik çekim eki
    "dı", "di", "du", "dü", "tı", "ti", "tu", "tü" # Katmerli birleşik çekim eki
]

print(len(set(ekler)))

203


In [49]:
edatlar = ["gibi", "için", "kadar", "ile", "üzere", "sanki", "den", "dolayı", "doğru", "önce", "sonra", "karşı", "göre", "rağmen", "beri", "üzere", "yönelik", "ait"]
bağlaçlar = ["ve", "ama", "ile", "de", "çünkü", "eğer", "fakat", "lakin", "ki", "ya", "yahut", "ya da", "veya", "hem", "oysa", "oysa ki", "nitekim", "sanki", "zira", "öyle ki", "şu var ki", "lâkin", "üstelik", "ne ... ne", "hem ... hem", "ister ... ister", "gerek ... gerek(se)", "ancak", "bari"]
ünlemler = ["ah", "ey", "oha", "öf", "hah", "ya", "aman", "hey", "hadi", "vah", "eyvah", "uff", "of", "oh", "heyecanla", "tamam", "çüş"]
cekim_ekleri = ["ler", "lar", "i", "e", "den", "de", "im", "in", "i", "imiz", "iniz", "leri", "ın", "in", "un", "ün", "ca", "ce", "le", "la", "r", "yor", "mekte", "di", "miş", "ecek", "meli", "malı", "se", "sa", "ayım", "ydi", "ymiş", "yse", "dir", "m", "n", "k", "iz", "siniz", "ler", "mak", "mek"]
yapim_ekleri = ["lık", "li", "sız", "ci", "ce", "daş", "üncü", "msı", "cık", "tı", "cıl", "deki", "la", "al", "l", "a", "ar", "da", "ık", "ımsa", "laş", "sa", "ca", "acak", "ak", "ge", "gı", "ı", "ıcı", "ık", "ım", "nç", "ıntı", "ır", "ış", "ma", "dır", "l", "n", "t"]

# doldurulacaklar = predefined_words + edatlar + bağlaçlar + ünlemler + cekim_ekleri + yapim_ekleri
# TypeError: can only concatenate list (not "tuple") to list
doldurulacaklar = set()
doldurulacaklar.update(predefined_words)
doldurulacaklar.update(edatlar)
doldurulacaklar.update(bağlaçlar)
doldurulacaklar.update(ünlemler)
doldurulacaklar.update(cekim_ekleri)
doldurulacaklar.update(yapim_ekleri)
doldurulacaklar.update(ekler)
doldurulacaklar = list(doldurulacaklar)
len(doldurulacaklar)

25171

In [ ]:
25072

In [50]:
tokenizer = Tokenizer(BPE(unk_token=unk_token))
tokenizer.pre_tokenizer = Whitespace()
tokenizer.add_tokens(list(doldurulacaklar))
tokenizer

Tokenizer(version="1.0", truncation=None, padding=None, added_tokens=[{"id":0, "content":"tahsisat", "single_word":False, "lstrip":False, "rstrip":False, "normalized":True, "special":False}, {"id":1, "content":"Yurt", "single_word":False, "lstrip":False, "rstrip":False, "normalized":True, "special":False}, {"id":2, "content":"Bulun", "single_word":False, "lstrip":False, "rstrip":False, "normalized":True, "special":False}, {"id":3, "content":"silsile", "single_word":False, "lstrip":False, "rstrip":False, "normalized":True, "special":False}, {"id":4, "content":"eflatun", "single_word":False, "lstrip":False, "rstrip":False, "normalized":True, "special":False}, {"id":5, "content":"tulumba", "single_word":False, "lstrip":False, "rstrip":False, "normalized":True, "special":False}, {"id":6, "content":"Hapishane", "single_word":False, "lstrip":False, "rstrip":False, "normalized":True, "special":False}, {"id":7, "content":"berki", "single_word":False, "lstrip":False, "rstrip":False, "normalized

In [52]:
trainer = BpeTrainer(special_tokens=spl_tokens, vocab_size=7001, limit_alphabet=1200, min_frequency=16)

trainer

BpeTrainer(BpeTrainer(min_frequency=16, vocab_size=7001, show_progress=True, special_tokens=[AddedToken(content="<bos>", single_word=False, lstrip=False, rstrip=False, normalized=False, special=True), AddedToken(content="<eos>", single_word=False, lstrip=False, rstrip=False, normalized=False, special=True), AddedToken(content="<unk>", single_word=False, lstrip=False, rstrip=False, normalized=False, special=True), AddedToken(content="<pad>", single_word=False, lstrip=False, rstrip=False, normalized=False, special=True), AddedToken(content="<start_of_turn>", single_word=False, lstrip=False, rstrip=False, normalized=False, special=True), AddedToken(content="<end_of_turn>", single_word=False, lstrip=False, rstrip=False, normalized=False, special=True)], limit_alphabet=1200, initial_alphabet=[], continuing_subword_prefix=None, end_of_word_suffix=None, max_token_length=None, words={}))

In [10]:
from datasets import load_dataset

ds = load_dataset("wikimedia/wikipedia", "20231101.tr")
df = ds['train'].to_pandas()
df

,id,url,title,text
0,10,https://tr.wikipedia.org/wiki/Cengiz%20Han,Cengiz Han,"Cengiz Han (doğum adıyla Temuçin, – 18 Ağusto..."
1,16,https://tr.wikipedia.org/wiki/Film%20%28anlam%...,Film (anlam ayrımı),Film şu anlamlara gelebilir:\n\n Camlara yapış...
2,22,https://tr.wikipedia.org/wiki/Mustafa%20Suphi,Mustafa Suphi,"Mehmed Mustafa Subhi (), kısaca Mustafa Suphi,..."
3,24,https://tr.wikipedia.org/wiki/Linux,Linux,Linux (telaffuz: Lin-uks); Linux çekirdeğine d...
4,30,https://tr.wikipedia.org/wiki/Bol%C5%9Fevizm,Bolşevizm,"Bolşevik, çoğunluktan yana anlamına gelen Rusç..."
...,...,...,...,...
534983,3611624,https://tr.wikipedia.org/wiki/Musculus%20ptery...,Musculus pterygoideus lateralis,ağzı kapatan tek kastır. alt çene ramus'a ve ü...
534984,3611625,https://tr.wikipedia.org/wiki/%C3%87in%20kad%C...,Çin kadın millî futbol takımı,"Çin kadın millî futbol takımı, Çin'i uluslarar..."
534985,3611628,https://tr.wikipedia.org/wiki/Abdurrezak%20KUR...,Abdurrezak KURTULUŞ,1933 yılında Mardin’de doğdu. 1950 yılında Diy...
534986,3611631,https://tr.wikipedia.org/wiki/%C4%B0brahim%20T...,İbrahim Turgut,İbrahim Turgut Sayın Cumhurbaşkanı Recep Tayyi...


In [11]:
texts_with_title = df['title'] + " " + df['text']
texts = texts_with_title.to_list()
len(texts)

534988

In [53]:
tokenizer.train_from_iterator(texts, trainer=trainer)
tokenizer.save("custom_bpe_tokenizer.json")
tokenizer.get_vocab_size()


30164

In [54]:
from transformers import PreTrainedTokenizerFast

fast_tokenizer = PreTrainedTokenizerFast(tokenizer_file="custom_bpe_tokenizer.json")


/Users/alibayram/Library/Python/3.9/lib/python/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [55]:
encoded = tokenizer.encode("akademi akademisyen akademisyenler ve aileleri ile birlikte aktif çalışıyorlar")
print(tokenizer.decode(encoded.ids))
print(fast_tokenizer.tokenize("akademi akademisyen akademisyenler ve aileleri ile birlikte aktif çalışıyorlar"))

akademi akademisyen akademisyen ler ve aile leri ile birlikte aktif çalış ı yor lar
['akademi', 'akademisyen', 'akademisyen', 'ler', 've', 'aile', 'leri', 'ile', 'birlikte', 'aktif', 'çalış', 'ı', 'yor', 'lar']


In [ ]:
""" 
akademisyen Ada er ve aile Ada er ad il e birlikte aktif çalış Ahize yor la Adalet
['akademisyen', 'l', 'er', 've', 'aile', 'l', 'er', 'i', 'il', 'e', 'birlikte', 'aktif', 'çalış', 'ı', 'yor', 'la', 'r']

akade mi akademisyen akademisyenler ve aileleri ile birlikte aktif çalışıyor lar
['akade', 'mi', 'akademisyen', 'akademisyenler', 've', 'aileleri', 'ile', 'birlikte', 'aktif', 'çalışıyor', 'lar']
 """

In [56]:
fast_tokenizer.push_to_hub("alibayram/tr_tokenizer")

CommitInfo(commit_url='https://huggingface.co/alibayram/tr_tokenizer/commit/9c11d1b0ebe5e4b44604caff7ea7a9b1d831a42b', commit_message='Upload tokenizer', commit_description='', oid='9c11d1b0ebe5e4b44604caff7ea7a9b1d831a42b', pr_url=None, pr_revision=None, pr_num=None)

In [57]:
fast_tokenizer.tokenize(texts[0])

['C',
 'en',
 'giz',
 'Han',
 'C',
 'en',
 'giz',
 'Han',
 '(',
 'doğum',
 'ad',
 'ı',
 'y',
 'la',
 'Tem',
 'uç',
 'in',
 ',',
 '–',
 '18',
 'Ağustos',
 '12',
 '27',
 '),',
 'M',
 'o',
 'ğ',
 'ol',
 'İ',
 'm',
 'para',
 'tor',
 'lu',
 'ğ',
 'u',
 "'",
 'nun',
 'kur',
 'u',
 'cu',
 'su',
 've',
 'ilk',
 'Kağan',
 'ı',
 'ol',
 'an',
 'M',
 'o',
 'ğ',
 'ol',
 'komutan',
 've',
 'hükümdar',
 'dır',
 '.',
 'Hükümdar',
 'lı',
 'ğ',
 'ı',
 'dönem',
 'in',
 'de',
 'gerçekleş',
 'tir',
 'di',
 'ğ',
 'i',
 'hiçbir',
 'savaş',
 'ı',
 'kaybet',
 'mey',
 'en',
 'C',
 'en',
 'giz',
 'Han',
 ',',
 'dünya',
 'tarihi',
 'nin',
 'en',
 'büyük',
 'askeri',
 'lider',
 'leri',
 'n',
 'den',
 'biri',
 'si',
 'ol',
 'arak',
 'kabul',
 'e',
 'dil',
 'mekte',
 'dir',
 '.',
 '13',
 '.',
 'yüzyıl',
 'ın',
 'baş',
 'ın',
 'da',
 'Orta',
 'As',
 'ya',
 "'",
 'da',
 'ki',
 'tüm',
 'göçebe',
 'bozkır',
 'kavim',
 'leri',
 'n',
 'i',
 'birleş',
 'tir',
 'ip',
 'bir',
 'ulus',
 'hâ',
 'li',
 'ne',
 'getir',
 'erek',


In [4]:
# Load model directly
from transformers import AutoTokenizer

a_tokenizer = AutoTokenizer.from_pretrained("alibayram/tr_tokenizer", use_fast=True)

In [ ]:
a_tokenizer.tokenize("Akademisyenler ve aileleri ile birlikte aktif çalışıyorlar")

['akademisyen',
 'ler',
 've',
 'aile',
 'leri',
 'ile',
 'birlikte',
 'aktif',
 'çalış',
 'ı',
 'yor',
 'lar']